In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Read original dataset
full_data = pd.read_csv("CC_LCL-FullData.csv")

# Convert datetime column to pandas DateTime type
full_data['DateTime'] = pd.to_datetime(full_data['DateTime'])

# Extract the year
full_data['year'] = full_data['DateTime'].dt.year

# Only use year 2013
full_data = full_data[full_data['year'] == 2013]
full_data.drop(columns=['year'], inplace=True)

# Extract day of the year
full_data['dayoftheyear'] = full_data['DateTime'].dt.dayofyear

# Check if the day is a weekend (Saturday=5, Sunday=6)
full_data['is_weekend'] = full_data['DateTime'].dt.dayofweek >= 5

# Extract hour
full_data['hour'] = full_data['DateTime'].dt.hour

# Drop DateTime column as it is not needed anymore
full_data.drop(columns=["DateTime"], inplace=True)

# Select only standard customers 
full_data = full_data[full_data["stdorToU"] == "Std"]
full_data.drop(columns=["stdorToU"], inplace=True)

# Encode the ID of meters
label_encoder_lclid = LabelEncoder()
full_data['LCLid'] = label_encoder_lclid.fit_transform(full_data['LCLid'])
full_data['LCLid'] = full_data['LCLid'].astype('int16')

# Ensure that target does not contain NULLs and is numerric 
full_data['KWH/hh (per half hour) '] = pd.to_numeric(full_data['KWH/hh (per half hour) '], errors='coerce').fillna(0.0).astype('float16')

# Calculate hourly consumption 
hourly_data = full_data.groupby(['LCLid','dayoftheyear','hour','is_weekend'])['KWH/hh (per half hour) '].sum().reset_index()
hourly_data = hourly_data.rename(columns={'KWH/hh (per half hour) ': 'KWH/hh (per hour) '})

# Sort the data 
hourly_data.sort_values(by=['LCLid', 'dayoftheyear', 'hour'], inplace=True)

# Save the DataFrame to a CSV file
hourly_data.to_csv('Preprocessed_data.csv', index=False)

In [2]:
hourly_data.tail()

,LCLid,dayoftheyear,hour,is_weekend,KWH/hh (per hour)
37078049,4410,365,19,False,0.226929
37078050,4410,365,20,False,0.112000
37078051,4410,365,21,False,0.104004
37078052,4410,365,22,False,0.100983
37078053,4410,365,23,False,0.119019
